In [1]:
from bs4 import BeautifulSoup as bs 
import requests

In [2]:
r = requests.get('https://en.wikipedia.org/wiki/Provinces_of_Cambodia')
soup = bs(r.content)

In [3]:
table = soup.select('.sortable tr')

In [4]:
from bs4 import Tag
provinces = []

title = ['Province_EN', 'Province_KH', 'Capital', 'Population(2019)','Area','Density','ISO']
for i in table[1:-1]:
    row = i.select('tr td')
    province=[]
    districts = []
    dist_district = {}
    for tag in row:
        for j in tag:
            
#           ==== Checking tag in data ====

            if j.nextSibling == None:
                province.append(j)
            else:
                for a in j:
                    if type(a) == Tag:
                        province.append(a.get_text())
                    else:
                        province.append(a)
                        
#   ==== getting link of province ====

    link = 'https://en.m.wikipedia.org/wiki/'+province[0].replace(' ','_') + '_province'
    req = requests.get(link)
    dis_soup = bs(req.content)
    dis_table = dis_soup.select('.wikitable tbody tr')
    
    title_district = ['ISO_Code','District_Name_EN','District_Name_KH','Population(2019)']
    
    
#   ==== Cleaning data ====    

    for dis in dis_table[1:-1]:
        district = []
        new_district = {}
        for j in dis.select('td'):
            district.append(j.get_text())
        for index,clean_n in enumerate(district):
            district[index] = clean_n.replace('\n','')
            
    
#   ==== adding title to data using dist ====

        for key_dist,data_dist in zip(title_district,district):
            new_district[key_dist] = data_dist
#         print(new_district)
        districts.append(new_district)
    
       
    
    
#   ==== insert all data to provinces list ====     
        
#   ==== remove '\n' item from list ==== 

    for clean in province:
        if clean == '\n':
            province.remove(clean)

#   ==== replace '\n' in text ====

    for index,text_replace in enumerate(province):
        province[index] = text_replace.replace('\n','').replace(' Municipality','')
        
#   ==== adding title to data using dist ====
    new_province_list = {}
    for key, data in zip(title,province):
        new_province_list[key]=data
#     print(new_province_list)
    new_province_list['District'] = districts
    provinces.append(new_province_list)
#     print(dist_district)
# print(provinces)

In [392]:
import json 

In [393]:
with open('cambodia.json','w',encoding='utf-8') as f:
    json.dump(provinces, f, ensure_ascii=False, indent=2)